<a href="https://colab.research.google.com/github/Abhijth01/Multi-model-Ai-Agent/blob/main/MULTI_MODEL_RESEARCH_AI_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. Model Configuration ---
import google.generativeai as genai
from google.generativeai import types

def configure():
    """
    Configures the Gemini API client securely using Colab Secrets or environment variables.
    """
    secret_name = "GEMINI_API_KEY"

    # Try retrieving API key from Colab Secrets, else from environment
    try:
        from google.colab import userdata
        api_key = userdata.get(secret_name)
    except ImportError:
        import os
        api_key = os.environ.get(secret_name)

    if not api_key:
        raise ValueError(f"Secret '{secret_name}' not found. Please check your environment variables or Colab Secrets.")

    # Configure the Gemini client
    genai.configure(api_key=api_key)
    print("✅ Gemini Model configured successfully.")

    # Return an initialized GenerativeModel instance
    # 'gemini-2.0-flash' or 'gemini-1.5-flash' are most common and stable model names
    return genai.GenerativeModel(model_name="gemini-2.0-flash")


In [2]:
import ast
import google.generativeai as genai

def planner_agent(model, topic: str) -> list[str]:
    """
    Planner Agent: Breaks down a research topic into 3-5 specific questions.
    Handles Markdown formatting in the model response.
    """
    print("🧠 Planner Agent: Creating a research plan...")

    prompt = f"""
    You are an expert research planner.
    Break down the following topic into 3–5 clear, specific, and answerable questions.
    Return ONLY a valid Python list of strings — no explanations, no extra text.

    TOPIC: "{topic}"

    Example output format:
    ["question 1", "question 2", "question 3", "question 4"]
    """

    try:
        # Pass contents as a list (required by Gemini SDK)
        response = model.generate_content(contents=[prompt])
        plan_str = (response.text or "").strip()

        # Remove triple backticks or Markdown formatting
        plan_str = plan_str.replace("```python", "").replace("```", "").strip()

        # Attempt to safely parse as Python list
        plan = ast.literal_eval(plan_str)

        # Validate output type
        if not isinstance(plan, list) or not all(isinstance(item, str) for item in plan):
            raise ValueError("Model output is not a valid list of strings.")

        print("✅ Plan created:")
        for i, q in enumerate(plan, 1):
            print(f"   {i}. {q}")
        return plan

    except (ValueError, SyntaxError) as e:
        # Fallback parsing if literal_eval fails
        print(f"⚠️ Error parsing output: {e}")
        print(f"   Raw model output: {plan_str}")
        fallback = plan_str.replace('[', '').replace(']', '').replace('"', '').replace("'", '')
        plan = [q.strip() for q in fallback.split(',') if q.strip()]

        if plan:
            print("✅ Fallback plan created:")
            for i, q in enumerate(plan, 1):
                print(f"   {i}. {q}")
        else:
            print("❌ Failed to recover a valid plan.")
        return plan
    except Exception as e:
        print(f"❌ Error in Planner Agent: {e}")
        return []


In [3]:
def search_agent(model, question: str) -> str:
    """
    Search Agent: Queries the Gemini model directly for answers.
    Avoids unavailable tool classes for current SDK versions.
    """
    print(f"🔍 Search Agent: Researching question: '{question}'...")
    try:
        # Direct query (no tools, compatible with current Gemini SDK)
        response = model.generate_content(contents=[question])
        if hasattr(response, "text") and response.text:
            print("   - ✅ Information found.")
            return response.text.strip()
        else:
            print("   - ⚠️ No text returned from model.")
            return ""
    except Exception as e:
        print(f"❌ Error in Search Agent: {e}")
        return ""


In [4]:
# --- 4. Synthesizer Agent ---
from typing import List, Tuple
import google.generativeai as genai

# Define the type alias for clarity
ResearchResult = Tuple[str, str]

def synthesizer_agent(model, topic: str, research_results: List[ResearchResult]) -> str:
    """
    Synthesizes collected research data into a clear, well-structured report.
    """
    print("\n🧩 Synthesizer Agent: Writing the final report...")

    # ✅ FIX 1: Validate that we actually received research results
    if not research_results:
        print("⚠️ No research data provided to synthesize.")
        return "No research data provided to synthesize."

    # ✅ FIX 2: Use consistent, structured formatting for clarity
    research_notes = "\n".join(
        f"### Question:\n> {question}\n\n### Research Data:\n{data}\n\n---"
        for question, data in research_results
    )

    # ✅ FIX 3: Well-structured prompt with clear instructions and placeholders
    prompt = f"""
    You are an expert research analyst. Your task is to synthesize the provided research notes
    into a comprehensive, objective, and well-written report on the topic: "{topic}".

    Follow this structure:
    1. **Introduction** – Give a concise overview of the topic and context.
    2. **Body** – Combine the key insights from the questions and research notes into coherent paragraphs.
    3. **Conclusion** – Summarize the key findings and their implications.

    Important:
    - DO NOT copy or mention "Research Notes" or any headers like "Question:" or "Research Data:".
    - Write in an analytical, natural tone, as if preparing a research summary.
    - Keep it factual, cohesive, and professionally formatted.

    ## Research Notes ##
    {research_notes}
    """

    try:
        # ✅ FIX 4: Always pass `contents` as a list for compatibility
        response = model.generate_content(contents=[prompt])

        # ✅ FIX 5: Safely extract text
        if hasattr(response, "text") and response.text:
            print("✅ Final report successfully synthesized.")
            return response.text.strip()
        else:
            print("⚠️ No text returned from model.")
            return "Error: The model did not return any text."

    except Exception as e:
        print(f"❌ Error in Synthesizer Agent: {e}")
        return "Error: Could not generate the final report."


In [5]:
# --- 5. Orchestrator Function ---

from typing import List
import google.generativeai as genai

# Make sure these imports exist in your file:
# from your_module import configure, planner_agent, search_agent, synthesizer_agent, ResearchResult

def main():
    try:
        # 1. CONFIGURE MODEL
        model = configure()
    except ValueError as e:
        print(f"❌ Configuration Error: {e}")
        return
    except Exception as e:
        print(f"❌ Unexpected error during model configuration: {e}")
        return

    print("\n🤖 Hello! I am your AI Research Assistant.")
    topic = input("What topic would you like me to research today? ").strip()

    if not topic:
        print("⚠️ A topic is required to begin research. Exiting.")
        return

    print(f"\n🚀 Starting research process for: '{topic}'")

    # 2. PLAN
    research_plan = planner_agent(model, topic)
    if not research_plan:
        print("⚠️ Could not create a research plan. Exiting.")
        return

    # 3. SEARCH (Iterative)
    research_results: List[ResearchResult] = []
    for question in research_plan:
        print(f"\n🔎 Working on question: {question}")
        research_data = search_agent(model, question)
        if research_data:
            research_results.append((question, research_data))
        else:
            print("⚠️ No data found for this question.")

    if not research_results:
        print("⚠️ No useful research data found. Exiting.")
        return

    # 4. SYNTHESIZE
    final_report = synthesizer_agent(model, topic, research_results)

    # 5. OUTPUT
    print("\n\n📘 --- FINAL RESEARCH REPORT ---")
    print(f"## Topic: {topic}\n")
    print(final_report or "⚠️ No final report generated.")
    print("\n--- END OF REPORT ---")

if __name__ == "__main__":
    main()


✅ Gemini Model configured successfully.

🤖 Hello! I am your AI Research Assistant.
What topic would you like me to research today? use of smartphones in new gen

🚀 Starting research process for: 'use of smartphones in new gen'
🧠 Planner Agent: Creating a research plan...
✅ Plan created:
   1. How has smartphone usage impacted the academic performance of the new generation?
   2. What are the primary ways the new generation utilizes smartphones for social interaction and communication?
   3. To what extent does smartphone addiction affect the mental health and well-being of the new generation?
   4. How do socioeconomic factors influence the types of smartphones and apps used by the new generation?
   5. What is the role of smartphones in shaping the new generation's access to information and news?

🔎 Working on question: How has smartphone usage impacted the academic performance of the new generation?
🔍 Search Agent: Researching question: 'How has smartphone usage impacted the academic